# Foxglove Data Platform Demo Notebook

This notebook demonstrates retrieving data and events from Foxglove Data Platform.


In [ ]:
# First let's we can create an instance of our client.

from foxglove_data_platform.client import Client, OutputFormat

client = Client(token="fox_sk_CFlhbEgxMQbHBQGRCNE7c3CBaOxtrpk5")

In [ ]:
# Next let's use the client to list our imports.

import pandas

imports = sorted(client.get_imports(), key=lambda i: i["start"])
pandas.DataFrame(imports[0:3], columns=["device_id", "start", "end", "filename"])

In [ ]:
# Now let's download some GPS data.

gps_data = client.download_data(
    device_id=imports[1]["device_id"],
    start=imports[1]["start"],
    end=imports[1]["end"],
    topics=["/gps"],
    callback=lambda progress: print(".", end=""),
)

In [ ]:
# Let's read some messages from our mcap stream.

from io import BytesIO

import pandas
from mcap.mcap0.stream_reader import StreamReader
from mcap_ros1.ros1_decoder import Ros1Decoder

gps_reader = StreamReader(BytesIO(gps_data))
gps_decoder = Ros1Decoder(gps_reader)
gps_messages = [
    (message.latitude, message.longitude)
    for topic, record, message in gps_decoder.messages
]
pandas.DataFrame(gps_messages[0:3], columns=["lat", "lon"])

In [ ]:
# We can use our GPS messages to plot our course on the map.

import folium

map = folium.Map(location=gps_messages[0], zoom_start=200, width="50%")
folium.PolyLine(
    locations=gps_messages,
    weight=10,
    color="orange",
).add_to(map)
map

In [ ]:
# We can fetch acceleration data that spans multiple imports

imu_data = client.download_data(
    device_id=imports[0]["device_id"],
    start=imports[0]["start"],
    end=imports[-1]["end"],
    topics=["/imu"],
    callback=lambda progress: print(".", end=""),
)

imu_reader = StreamReader(BytesIO(imu_data))
imu_decoder = Ros1Decoder(imu_reader)
imu_messages = [
    {
        "time": pandas.Timestamp(message.header.stamp.to_nsec(), unit="ns").isoformat(),
        "accel_x": message.linear_acceleration.x,
        "accel_y": message.linear_acceleration.y,
    }
    for topic, record, message in imu_decoder.messages
]
imu_frame = pandas.DataFrame(imu_messages)

In [ ]:
imu_frame.plot(x="time", figsize=(10, 6), rot=45)